In [ ]:
import numpy as np
import torch

import json

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../')
import simulated_datasets_lib
import sdss_dataset_lib
import sdss_psf
import image_utils 

import starnet_vae_lib
import inv_kl_objective_lib as inv_kl_lib
import plotting_utils
import wake_sleep_lib

import psf_transform_lib
import image_statistics_lib

np.random.seed(34534)

In [ ]:
import fitsio

In [ ]:
psfield = fitsio.FITS('../../celeste_net/sdss_stage_dir/2583/2/136/psField-002583-2-0136.fit')

In [ ]:
psfield

In [ ]:
import psf_transform_lib2

In [ ]:
psf_params = torch.cat((psf_transform_lib2.get_psf_params(psfield, band = 2).unsqueeze(0), 
                       psf_transform_lib2.get_psf_params(psfield, band = 3).unsqueeze(0)))

In [ ]:
pl_psf = psf_transform_lib2.PowerLawPSF(init_psf_params=psf_params,
                                        normalization_constant=torch.ones(2), 
                                       psf_slen = 5)

In [ ]:
pl_psf.init_psf.shape

In [ ]:
plt.matshow(pl_psf.init_psf[0].detach())
plt.colorbar()

In [ ]:
plt.matshow(pl_psf.init_psf[1].detach())
plt.colorbar()

In [ ]:
# load my psf

In [ ]:
psf_dir = '../../multiband_pcat/Data/idR-002583-2-0136/psfs/'
psf_r = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-r.fits')[0].read()
psf_i = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-i.fits')[0].read()

psf_og = torch.Tensor(np.array([psf_r, psf_i]))


In [ ]:
psf_og.shape

In [ ]:
normalization_constant = simulated_datasets_lib._trim_psf(psf_og, 5).reshape(2, -1).sum(1) / pl_psf.init_psf.view(2, -1).sum(1)

In [ ]:
normalization_constant

In [ ]:
pl_psf_normed = \
    psf_transform_lib2.PowerLawPSF(init_psf_params=psf_params, 
                                   normalization_constant=normalization_constant, 
                                  psf_slen = 5)

In [ ]:
# compare

In [ ]:
plt.matshow(pl_psf_normed.init_psf[0].detach() - psf_og[0])
plt.colorbar()

In [ ]:
plt.matshow(pl_psf_normed.init_psf[1].detach() - psf_og[1])
plt.colorbar()

In [ ]:
# np.savetxt('../data/pl_psf_r.txt', pl_psf_normed.init_psf[0].detach().numpy())
# np.savetxt('../data/pl_psf_i.txt', pl_psf_normed.init_psf[1].detach().numpy())

In [ ]:
bands = [2, 3]

In [ ]:
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData(bands = bands, fudge_conversion = 1.0)

# image 
full_image = sdss_hubble_data.sdss_image.unsqueeze(0)
full_background = sdss_hubble_data.sdss_background.unsqueeze(0) * 0.0 + 823.


In [ ]:
pl_psf_normed.normalization_constant

In [ ]:
recon_mean, _ = \
        psf_transform_lib.get_psf_loss(full_image, 
                                        full_background,
                                        sdss_hubble_data.locs.unsqueeze(0), 
                                        sdss_hubble_data.fluxes.unsqueeze(0), 
                                        n_stars = torch.Tensor([len(sdss_hubble_data.fluxes)]).type(torch.long), 
                                        psf = pl_psf_normed.forward(),
                                        pad = 5)

In [ ]:
foo = (recon_mean[0, 0].detach() - full_image[0, 0]) / full_image[0, 0]
plt.matshow(foo, vmax = foo.abs().max(), vmin = -foo.abs().max(), cmap = plt.get_cmap('bwr'))
plt.colorbar()